##  Imports & GF(2) helpers

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def mod2(x):
    return (x % 2).astype(int) if isinstance(x, np.ndarray) else int(x % 2)

## Choose (n,k) and build a systematic generator G

In [9]:
def make_systematic_G(n=7, k=4, rng_seed=0):
    assert 0 < k < n, "k must be between 1 and n-1"
    rng = np.random.default_rng(rng_seed)
    I = np.eye(k, dtype=int)
    P = rng.integers(0, 2, size=(k, n-k), dtype=int)
    G = np.concatenate([I, P], axis=1)
    return G


# Example
n, k = 7, 4 # feel free to try other small sizes (e.g., n=10,k=5)
G = make_systematic_G(n, k, rng_seed=1)
print("G=\n", G)

G=
 [[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 0]
 [0 0 1 0 1 1 0]
 [0 0 0 1 0 1 0]]


## Derive parity-check H and verify G H^T ≡ 0

In [10]:
def make_H_from_G(G):
    k, n = G.shape
    P = G[:, k:]
    H = np.concatenate([P.T, np.eye(n-k, dtype=int)], axis=1)
    return H

    
H = make_H_from_G(G)
ok = np.all(mod2(G @ H.T) == 0)
print("H=\n", H)
print("Orthogonality G H^T == 0?", ok)

H=
 [[0 1 1 0 1 0 0]
 [1 0 1 1 0 1 0]
 [1 0 0 0 0 0 1]]
Orthogonality G H^T == 0? True


## Build a single-bit syndrome decoding table

In [11]:
def build_syndrome_table(H):
    n = H.shape[1]
    table = {tuple([0]*H.shape[0]): None} # no-error syndrome
    I = np.eye(n, dtype=int)
    for i in range(n):
        e = I[:, i]
        s = tuple(mod2(H @ e))
        table[s] = i
    return table

    
S_TABLE = build_syndrome_table(H)
print("#syndromes in table:", len(S_TABLE))

#syndromes in table: 6


## Encode, transmit over BSC, and decode

In [12]:
def encode_blocks(U, G):
    return mod2(U @ G)
    
def bsc(bits, e, rng=None):
    rng = np.random.default_rng(rng)
    flips = (rng.random(bits.shape) < e).astype(int)
    return mod2(bits + flips) # XOR

def decode_blocks(R, H, S_TABLE, k):
# R: (m,n), return estimated info bits (m,k)
    Uhat = []
    for r in R:
        s = tuple(mod2(H @ r))
        if s in S_TABLE and S_TABLE[s] is not None:
            i = S_TABLE[s] # bit to flip
            r = r.copy()
            r[i] ^= 1
        Uhat.append(r[:k])
    return np.array(Uhat, dtype=int)
    
# Quick sanity test on a few random blocks
rng = np.random.default_rng(0)
m = 8
U = rng.integers(0,2,size=(m,k))
V = encode_blocks(U, G)
R = V.copy()
# inject a single-bit error into each block at random position
for i in range(m):
    pos = rng.integers(0, n)
    R[i, pos] ^= 1
Uhat = decode_blocks(R, H, S_TABLE, k)
print("All corrected?", np.array_equal(Uhat, U))

All corrected? False


## BER experiment on a BSC (uncoded vs coded)

In [13]:
def ber(a, b):
    a = a.ravel(); b = b.ravel()
    return np.mean(a != b)

def simulate_BER(G, H, S_TABLE, nblocks=20000, e=0.05, rng_seed=123):
    rng = np.random.default_rng(rng_seed)
    k, n = G.shape[0], G.shape[1]
# random info bits
    U = rng.integers(0,2,size=(nblocks,k))
# uncoded baseline
    uncoded_tx = U.ravel()
    uncoded_rx = bsc(uncoded_tx, e, rng).reshape(-1,k)
    ber_uncoded = ber(U, uncoded_rx)
# coded
    V = encode_blocks(U, G)
    coded_rx = bsc(V, e, rng).reshape(-1,n)
    Uhat = decode_blocks(coded_rx, H, S_TABLE, k)
    ber_coded = ber(U, Uhat)
    return ber_uncoded, ber_coded

# Example
bu, bc = simulate_BER(G, H, S_TABLE, nblocks=10000, e=0.05, rng_seed=7)
print(f"Example e=0.05 -> BER_uncoded={bu:.5f}, BER_coded={bc:.5f}")

Example e=0.05 -> BER_uncoded=0.04940, BER_coded=0.03367


## Estimate minimum distance for small (n,k)

In [14]:
def min_distance_estimate(G, max_words=4096):
    k, n = G.shape[0], G.shape[1]
    limit = min(1<<k, max_words)
    dmin = n+1
    for u_int in range(1, limit): # skip all-zero
        u = np.array([(u_int>>i)&1 for i in range(k)][::-1], dtype=int)
        v = mod2(u @ G)
        w = int(v.sum())
        if 0 < w < dmin:
            dmin = w
    return dmin if dmin <= n else None

    
d_est = min_distance_estimate(G, max_words=1<<k if k<=12 else 4096)
print("Estimated d_min:", d_est)

Estimated d_min: 2
